In [5]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from kohonen import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
kohonen()

C:\Users\andre\Documents\EPFL\Boulot\NN\miniproject_1\src\kohonen.py:139: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return np.sort(x[t,:])


[3 4 8 9]


ValueError: num must be 1 <= num <= 36, not 0